In [86]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
Train=pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
Test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [87]:
#replace ".."with .100
Train=Train.replace('..', .100)  
Test=Test.replace('..', 0.100)  

In [88]:
#Unique country for iteration
unique = Train['Country Name'].unique()

In [89]:
#trainig and inferencing with LogisticRegression and ARIMA
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
j=0
for i in unique:
    transposed_df=0
    fg = Train[Train['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Code', 'Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    Xtrain = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
    Xtrain
    Ytrain=transposed_df[11]
    Ytrain
    logistic_regression = LogisticRegression()
    try:
        logistic_regression.fit(Xtrain,Ytrain)
        fgt = Test[Test['Country Name'] == i].reset_index(drop=True)
        columns_to_drop = [ 'Country Name','Indicator']
        fgt = fgt.drop(columns_to_drop, axis=1)
        transposed_df = fgt.transpose()
        #print(transposed_df)
  
        Xtest = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
        forecast = logistic_regression.predict(Xtest.astype(float))
        
        model = ARIMA(Ytrain.astype(float), order=(0,1,0))  # Adjust order as needed
        model_fit = model.fit()

    
        forecast2 = model_fit.forecast(steps=16)
        
        row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


        new_values = [i, forecast[0], forecast[1],forecast[2], forecast[3], forecast[4],forecast2[31]]
        sample_submission.loc[row_index] = new_values
    except:
        j+=1
    
    



/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, 

In [90]:
sample_submission

,Country Name,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
0,Afghanistan,0.298087813,0.298087813,0.298087813,0.298087813,0.298087813,0.297972
1,Albania,1.665218525,1.665218525,1.665218525,1.665218525,1.665218525,1.665219
2,Algeria,3.951960939,3.951960939,3.951960939,3.951960939,3.951960939,3.951961
3,American Samoa,3.5276,3.1392,1.1129,1.6479,3.1007,0.684000
4,Andorra,6.890389929,6.890389929,6.890389929,6.890389929,6.890389929,6.485769
...,...,...,...,...,...,...,...
261,Sub-Saharan Africa,0.76398389,0.76398389,0.76398389,0.76398389,0.76398389,0.763984
262,Sub-Saharan Africa (excluding high income),0.763559116,0.763559116,0.763559116,0.763559116,0.763559116,0.763559
263,Sub-Saharan Africa (IDA & IBRD countries),0.76398389,0.76398389,0.76398389,0.76398389,0.76398389,0.763984
264,Upper middle income,5.735859454,5.735859454,5.735859454,5.735859454,5.735859454,5.735859


In [91]:
sample_submission.to_csv('submission.csv', index=False)